In [ ]:
!pip install -q transformers torchaudio sentencepiece gtts huggingface_hub
!apt install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest

In [ ]:
import torch
import time
import re
import os
from transformers import pipeline, AutoModelForSpeechSeq2Seq, AutoProcessor
from gtts import gTTS
from IPython.display import Audio, display, HTML
import base64
from google.colab import output
from huggingface_hub import notebook_login

In [ ]:
# 1. Авторизация на Hugging Face
notebook_login('')  # Нажмите на ссылку и введите ваш токен

# 2. Инициализация моделей
device = "cuda" if torch.cuda.is_available() else "cpu"

# Модель для распознавания речи (Whisper)
asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-medium",
    device=device
)

# Русскоязычная модель для диалогов
chat_pipeline = pipeline(
    "text-generation",
    model="IlyaGusev/saiga_llama3_8b",
    device_map="auto",
    torch_dtype=torch.float16
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:38: FutureWarning: Deprecated positional argument(s) used in 'notebook_login': pass new_session='' as keyword args. From version 1.0 passing these as positional arguments will result in an error,
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Список для хранения истории диалога
chat_history = []

def process_audio(audio_b64):
    """Обработка аудиозаписи: распознавание речи, генерация ответа и озвучка"""
    try:
        # 1. Декодирование аудио
        audio_bytes = base64.b64decode(audio_b64)
        os.makedirs("/content/temp", exist_ok=True)
        webm_path = "/content/temp/input.webm"
        wav_path = "/content/temp/input.wav"

        with open(webm_path, "wb") as f:
            f.write(audio_bytes)

        # 2. Конвертация в WAV с проверкой
        if not os.path.exists(webm_path):
            raise FileNotFoundError(f"Файл {webm_path} не создан")

        !ffmpeg -y -i {webm_path} -acodec pcm_s16le -ar 16000 {wav_path} 2> /dev/null

        if not os.path.exists(wav_path):
            raise FileNotFoundError(f"Конвертация в WAV не удалась")

        # 3. Распознавание речи с использованием Whisper
        result = asr_pipeline(wav_path, generate_kwargs={"language": "russian"})
        user_text = result["text"]
        print("🎤 Вы сказали:", user_text)

        # 4. Добавляем сообщение пользователя в историю
        chat_history.append({"role": "user", "content": user_text})

        # Показываем сообщение пользователя сразу
        update_chat_interface()

        # 5. Формируем полный промпт с историей диалога
        system_prompt = """Ты — интеллектуальный виртуальный ассистент, специализированный на обучении русскому языку как родному или иностранному.
        Ты выполняешь роль внимательного, терпеливого и доброжелательного преподавателя, который помогает пользователю развивать навыки владения русским языком во всех аспектах:
грамматике, лексике, орфографии, пунктуации, фонетике (при необходимости), а также языковой культуре и стилю речи.
Основные принципы твоей работы:
Корректировка ошибок: при каждом высказывании пользователя внимательно отслеживай наличие ошибок. Исправляй их сразу, сохраняя исходную фразу и указывая, что было неверно.
После исправления поясняй правило, на котором основана корректировка, в доступной и понятной форме. Приводи минимум один пример правильного употребления.
Объяснение правил: всегда объясняй грамматические, орфографические и синтаксические правила, опираясь на нормы русского языка.
По возможности, используй краткие, запоминающиеся формулировки и приёмы (ассоциации, аналогии, таблицы).
Учитывай уровень пользователя: адаптируй объяснение под новичка или более продвинутого ученика.
Тон общения: общайся с пользователем в уважительной, доброжелательной и поддерживающей манере.
Поощряй стремление к изучению языка, даже если в сообщении пользователя много ошибок.
Используй стиль «мягкого наставника»: не критикуй, а мягко направляй и вдохновляй.
Активное сопровождение обучения: при необходимости предлагай короткие задания, упражнения, примеры.
Можешь использовать диалоговые практики, проверочные вопросы или мини-игры для закрепления материала."""

        # Создаем промпт с историей диалога
        prompt = f"<|start_header_id|>system<|end_header_id|>\n{system_prompt}<|eot_id|>\n"

        # Добавляем историю диалога (последние 10 сообщений для ограничения длины)
        recent_history = chat_history[-10:]  # Берем последние 10 сообщений
        for message in recent_history:
            if message["role"] == "user":
                prompt += f"<|start_header_id|>user<|end_header_id|>\n{message['content']}<|eot_id|>\n"
            else:
                prompt += f"<|start_header_id|>assistant<|end_header_id|>\n{message['content']}<|eot_id|>\n"

        prompt += "<|start_header_id|>assistant<|end_header_id|>\n"

        start_time = time.time()
        # 6. Генерация ответа с помощью русскоязычной LLM
        answer = chat_pipeline(
          prompt,
          max_new_tokens=512,
          do_sample=True,
          temperature=0.7,
          top_p=0.9,
          pad_token_id=chat_pipeline.tokenizer.eos_token_id
        )[0]['generated_text'].split('<|start_header_id|>assistant<|end_header_id|>')[-1].strip()

        end_time = time.time()
        generate_time = end_time - start_time
        print(f"Время генерации ответа: {generate_time:.2f} сек.")

        # Очищаем от специальных токенов
        clean_answer = re.sub(r'<\|.*?\|>', '', answer).replace('\n', ' ').strip()

        print("🤖 Ответ помощника:", clean_answer)

        # 7. Добавляем очищенный ответ ассистента в историю
        chat_history.append({"role": "assistant", "content": clean_answer})

        # 8. Обновляем интерфейс чата
        update_chat_interface()

        # 9. Синтез речи с использованием gTTS
        tts = gTTS(clean_answer, lang='ru', slow=False)
        tts.save("/content/temp/answer.mp3")
        display(Audio("/content/temp/answer.mp3", autoplay=True))
    except Exception as e:
        error_message = f"Извините, произошла ошибка: {str(e)}"
        print("⚠️ Ошибка обработки:", e)

        # Добавляем сообщение об ошибке в чат
        chat_history.append({"role": "assistant", "content": error_message})
        update_chat_interface()

        # Голосовое сообщение об ошибке
        error_tts = gTTS("Извините, произошла ошибка. Попробуйте еще раз.", lang='ru')
        error_tts.save("/content/temp/error.mp3")
        display(Audio("/content/temp/error.mp3", autoplay=True))
    finally:
        # Очистка временных файлов
        for f in [webm_path, wav_path]:
            if os.path.exists(f):
                os.remove(f)

def update_chat_interface():
    """Обновление интерфейса чата с историей диалога"""
    chat_html = ""
    for message in chat_history:
        if message["role"] == "user":
            chat_html += f"""
            <div style="display: flex; justify-content: flex-end; margin: 10px 0;">
                <div style="
                    max-width: 70%;
                    background: #007bff;
                    color: white;
                    padding: 12px 16px;
                    border-radius: 18px 18px 4px 18px;
                    box-shadow: 0 2px 8px rgba(0,123,255,0.2);
                ">
                    <div style="font-size: 12px; opacity: 0.8; margin-bottom: 4px;">👤 Вы</div>
                    <div>{message['content']}</div>
                </div>
            </div>
            """
        else:
            chat_html += f"""
            <div style="display: flex; justify-content: flex-start; margin: 10px 0;">
                <div style="
                    max-width: 70%;
                    background: #f8f9fa;
                    color: #333;
                    padding: 12px 16px;
                    border-radius: 18px 18px 18px 4px;
                    border: 1px solid #e9ecef;
                    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
                ">
                    <div style="font-size: 12px; color: #666; margin-bottom: 4px;">🤖 Помощник</div>
                    <div>{message['content']}</div>
                </div>
            </div>
            """

    # Обновляем содержимое чата
    display(HTML(f"""
    <script>
        const chatContainer = document.getElementById('chat-messages');
        if (chatContainer) {{
            chatContainer.innerHTML = `{chat_html}`;
            chatContainer.scrollTop = chatContainer.scrollHeight;
        }}
    </script>
    """))

def clear_chat_history():
    """Очистка истории чата"""
    global chat_history
    chat_history = []
    update_chat_interface()
    print("🧹 История чата очищена")

# Создание интерфейса чата
display(HTML("""
<div style="font-family: 'Segoe UI', Arial, sans-serif; max-width: 800px; margin: 20px auto; padding: 20px; border-radius: 12px; background: #ffffff; box-shadow: 0 4px 20px rgba(0,0,0,0.1);">

    <!-- Заголовок -->
    <div style="text-align: center; margin-bottom: 20px; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 10px; color: white;">
        <h2 style="margin: 0 0 10px 0;">🎓 Виртуальный помощник для изучения русского языка</h2>
        <p style="margin: 0; opacity: 0.9;">Говорите на русском языке и получайте помощь в обучении</p>
    </div>

    <!-- Область чата -->
    <div style="
        border: 2px solid #e9ecef;
        border-radius: 12px;
        height: 400px;
        overflow-y: auto;
        padding: 20px;
        background: #fafbfc;
        margin-bottom: 20px;
        position: relative;
    " id="chat-messages">
        <div style="text-align: center; color: #6c757d; padding: 50px 20px;">
            <div style="font-size: 48px; margin-bottom: 16px;">💬</div>
            <div style="font-size: 18px; margin-bottom: 8px;">Добро пожаловать!</div>
            <div style="font-size: 14px;">Нажмите "Начать запись" и начните говорить</div>
        </div>
    </div>

    <!-- Панель управления -->
    <div style="display: flex; justify-content: center; align-items: center; gap: 15px; flex-wrap: wrap;">
        <button id="startBtn" style="
            padding: 15px 30px;
            font-size: 16px;
            background: #28a745;
            color: white;
            border: none;
            border-radius: 25px;
            cursor: pointer;
            box-shadow: 0 4px 12px rgba(40,167,69,0.3);
            transition: all 0.3s ease;
            font-weight: 600;
        ">
            🎤 Начать запись
        </button>

        <button id="stopBtn" style="
            padding: 15px 30px;
            font-size: 16px;
            background: #dc3545;
            color: white;
            border: none;
            border-radius: 25px;
            cursor: pointer;
            box-shadow: 0 4px 12px rgba(220,53,69,0.3);
            transition: all 0.3s ease;
            display: none;
            font-weight: 600;
        ">
            ⏹️ Завершить запись
        </button>

        <button id="clearBtn" style="
            padding: 12px 20px;
            font-size: 14px;
            background: #6c757d;
            color: white;
            border: none;
            border-radius: 20px;
            cursor: pointer;
            box-shadow: 0 2px 8px rgba(108,117,125,0.3);
            transition: all 0.3s ease;
        " onclick="google.colab.kernel.invokeFunction('clear_chat_history', [], {});">
            🧹 Очистить чат
        </button>
    </div>

    <!-- Статус и таймер -->
    <div style="text-align: center; margin-top: 20px;">
        <div id="status" style="margin: 10px 0; color: #666; font-size: 14px;"></div>
        <div id="timer" style="font-size: 20px; margin: 10px; color: #007bff; font-weight: bold;"></div>
    </div>

    <!-- Подсказка -->
    <div style="margin-top: 20px; padding: 15px; background: #e7f3ff; border-radius: 8px; border-left: 4px solid #007bff;">
        <div style="color: #0056b3; font-weight: 600; margin-bottom: 5px;">💡 Как пользоваться:</div>
        <div style="color: #0056b3; font-size: 14px;">
            1. Нажмите "Начать запись"<br>
            2. Говорите на русском языке<br>
            3. Нажмите "Завершить запись"<br>
            4. Получите ответ и продолжите диалог
        </div>
    </div>
</div>

<script>
const startBtn = document.getElementById("startBtn");
const stopBtn = document.getElementById("stopBtn");
const status = document.getElementById("status");
const timer = document.getElementById("timer");

let mediaRecorder = null;
let recordingChunks = [];
let startTime = null;
let timerInterval = null;

// Функция обновления таймера
function updateTimer() {
    if (startTime) {
        const elapsed = Math.floor((Date.now() - startTime) / 1000);
        const minutes = Math.floor(elapsed / 60);
        const seconds = elapsed % 60;
        timer.textContent = `⏱️ Запись: ${minutes}:${seconds.toString().padStart(2, '0')}`;
    }
}

// Добавляем эффекты наведения для кнопок
startBtn.onmouseover = () => startBtn.style.transform = "translateY(-2px)";
startBtn.onmouseout = () => startBtn.style.transform = "translateY(0)";
stopBtn.onmouseover = () => stopBtn.style.transform = "translateY(-2px)";
stopBtn.onmouseout = () => stopBtn.style.transform = "translateY(0)";

// Начало записи
startBtn.onclick = async function() {
    try {
        status.textContent = "🔍 Получение доступа к микрофону...";

        const stream = await navigator.mediaDevices.getUserMedia({
            audio: {
                sampleRate: 16000,
                channelCount: 1,
                echoCancellation: true,
                noiseSuppression: true
            }
        });

        recordingChunks = [];
        mediaRecorder = new MediaRecorder(stream, {
            mimeType: 'audio/webm;codecs=opus'
        });

        mediaRecorder.ondataavailable = (event) => {
            if (event.data.size > 0) {
                recordingChunks.push(event.data);
            }
        };

        mediaRecorder.onstop = async () => {
            stream.getTracks().forEach(track => track.stop());

            if (recordingChunks.length > 0) {
                status.textContent = "🔄 Обработка записи...";
                const audioBlob = new Blob(recordingChunks, { type: 'audio/webm' });

                const reader = new FileReader();
                reader.onload = () => {
                    const audioData = reader.result.split(',')[1];
                    google.colab.kernel.invokeFunction('process_audio', [audioData], {});
                    status.textContent = "✅ Готово! Можете записать новый вопрос";
                    timer.textContent = "";
                };
                reader.readAsDataURL(audioBlob);
            }
        };

        // Запуск записи
        mediaRecorder.start();
        startTime = Date.now();
        timerInterval = setInterval(updateTimer, 1000);

        startBtn.style.display = "none";
        stopBtn.style.display = "inline-block";
        status.textContent = "🔴 Идет запись... Говорите сейчас";

    } catch (error) {
        status.textContent = "❌ Ошибка доступа к микрофону: " + error.message;
        console.error("Ошибка:", error);
    }
};

// Остановка записи
stopBtn.onclick = function() {
    if (mediaRecorder && mediaRecorder.state === "recording") {
        mediaRecorder.stop();
        clearInterval(timerInterval);

        startBtn.style.display = "inline-block";
        stopBtn.style.display = "none";
        status.textContent = "⏳ Запись завершена. Обработка...";
    }
};

// Сброс интерфейса при ошибках
window.addEventListener('error', function() {
    startBtn.style.display = "inline-block";
    stopBtn.style.display = "none";
    clearInterval(timerInterval);
    timer.textContent = "";
});

status.textContent = "🟢 Система готова к работе!";
</script>
"""))

# Регистрация callback-функций
output.register_callback('process_audio', process_audio)
output.register_callback('clear_chat_history', clear_chat_history)

print("✅ Интеллектуальный голосовой чат-помощник готов к работе!")
print("🎯 Новые возможности:")
print("   • Интерфейс в виде чата с историей диалога")
print("   • Контекст предыдущих сообщений передается модели")
print("   • Визуальное разделение сообщений пользователя и помощника")
print("   • Возможность очистки истории чата")
print("   • Автопрокрутка к новым сообщениям")
print("\n💡 Теперь ваш голос отправляется в модель вместе с историей диалога!")

✅ Интеллектуальный голосовой чат-помощник готов к работе!
🎯 Новые возможности:
   • Интерфейс в виде чата с историей диалога
   • Контекст предыдущих сообщений передается модели
   • Визуальное разделение сообщений пользователя и помощника
   • Возможность очистки истории чата
   • Автопрокрутка к новым сообщениям

💡 Теперь ваш голос отправляется в модель вместе с историей диалога!


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=russian, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=russian.


🎤 Вы сказали:  Хочу домой.


Время генерации ответа: 305.71 сек.
🤖 Ответ помощника: Хочу домой - это правильный запрос. Однако стоит заметить, что в русском языке чаще используется форма "хочу домой", где слово "дома" уже подразумевается.  Правильная форма: **Хочу домой**.  Ошибки: 1. В данном случае нет явных ошибок в словах, но важно помнить о том, что в русской речи часто используются сочетания типа "хочу дома", где слово "дома" является прямым дополнением к глаголу "хочу".   Если вам нужен уточнительный вариант, я могу предложить его:  **Вариант 2:** Хочу назад домой  Этот вариант добавляет уточнение времени и места назначения ("назад"), что делает выражение более конкретным.  Надеюсь, это поможет! Если у вас есть ещё какие-то вопросы по русскому языку, пожалуйста, спрашивайте.
